In [17]:
import os
import shutil
import time
import threading
import urllib.parse
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, count

In [ ]:
import os
import shutil
import time
import threading
import urllib.parse
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, count

In [18]:
# === PAS 1: Setare corectă pentru Windows + spații în căi ===
warehouse_path = os.path.abspath("spark-warehouse").replace("\\", "/")
warehouse_uri = "file:///" + urllib.parse.quote(warehouse_path)

# === PAS 2: Creează și golește folderul de input ===
folder = "stream_input"
os.makedirs(folder, exist_ok=True)
for f in os.listdir(folder):
    path = os.path.join(folder, f)
    if os.path.isfile(path):
        os.remove(path)

# === PAS 3: Creează fișier dummy cu headerul (CSV valid) ===
with open(os.path.join(folder, "dummy.csv"), "w", encoding="utf-8") as f:
    f.write("Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Summary,Text,ReadableTime\n")

print("✅ dummy.csv creat în:", os.path.abspath(folder))

# === PAS 4: Inițializează SparkSession cu cale warehouse explicită ===
spark = SparkSession.builder \
    .appName("StreamingReviewStats") \
    .config("spark.sql.warehouse.dir", warehouse_uri) \
    .getOrCreate()

# === PAS 5: Definește schema ===
schema = "Id INT, ProductId STRING, UserId STRING, ProfileName STRING, HelpfulnessNumerator INT, HelpfulnessDenominator INT, Score INT, Summary STRING, Text STRING, ReadableTime STRING"

# === PAS 6: Inițializează streaming din folder ===
stream_df = spark.readStream \
    .schema(schema) \
    .option("header", True) \
    .csv(folder)

# === PAS 7: Agregare streaming ===
aggregated = stream_df.groupBy().agg(
    count("*").alias("nr_recenzii"),
    avg("Score").alias("scor_mediu")
)

query = aggregated.writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("truncate", False) \
    .start()

# === PAS 8: Thread pentru copiere fișier real după 3 secunde ===
def adauga_fisier_recenzii():
    time.sleep(3)
    shutil.copy("date/recenzii.csv", os.path.join(folder, "recenzii_1.csv"))
    print("✅ Fișier recenzii_1.csv adăugat în stream_input")

threading.Thread(target=adauga_fisier_recenzii).start()

# === PAS 9: Rămâne ascultând evenimente noi ===
query.awaitTermination()

✅ dummy.csv creat în: d:\Master\Sem II\BigData\ProiectCod\stream_input


Py4JJavaError: An error occurred while calling o131.csv.
: java.lang.UnsatisfiedLinkError: 'boolean org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(java.lang.String, int)'
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:793)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1249)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1454)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:761)
	at org.apache.spark.util.HadoopFSUtils$.listLeafFiles(HadoopFSUtils.scala:180)
	at org.apache.spark.util.HadoopFSUtils$.$anonfun$parallelListLeafFilesInternal$1(HadoopFSUtils.scala:95)
	at scala.collection.TraversableLike.$anonfun$